## Foundations of Data Science: Computational Thinking with Python

Mirroring course Jupyter notebook except using standard modules instead of Berkeley data science module.

## Lecture 12: Table Review 

Quick review of table methods

In [2]:
import numpy as np
import pandas as pd

pd.set_option('max_rows', 9)

In [3]:
# Create a table from rows
drinks = pd.DataFrame([['Milk Tea', 'Tea One', 4],
                       ['Espresso', 'Nefeli',  2],
                       ['Coffee',    'Nefeli', 3],
                       ['Espresso', "Abe's",   2]],
                      columns=['Drink', 'Cafe', 'Price'])
drinks

,Drink,Cafe,Price
0,Milk Tea,Tea One,4
1,Espresso,Nefeli,2
2,Coffee,Nefeli,3
3,Espresso,Abe's,2


In [6]:
# Create a table from columns
discounts = pd.DataFrame(
    {'Coupon % off':np.array([5, 50, 25]),
     'Location':np.array(['Tea One', 'Nefeli', 'Tea One'])})
discounts

,Coupon % off,Location
0,5,Tea One
1,50,Nefeli
2,25,Tea One


In [33]:
# Join Drinks to Discounts
#  only keep drinks with discounts 
drinks_discounted = drinks.join(discounts.set_index('Location'), 
                                on='Cafe', how='right')

drinks_discounted['Discounted_Price'] = (drinks_discounted['Price'] * 
                            (1 - drinks_discounted['Coupon % off'] / 100))
drinks_discounted.drop(['Price', 'Coupon % off'], axis='columns', inplace=True)
drinks_discounted.reset_index(drop=True, inplace=True)
drinks_discounted

,Drink,Cafe,Discounted_Price
0,Milk Tea,Tea One,3.8
1,Milk Tea,Tea One,3.0
2,Espresso,Nefeli,1.0
3,Coffee,Nefeli,1.5


In [58]:
# List of unique drinks
drinks_discounted.Drink.unique()

array(['Milk Tea', 'Espresso', 'Coffee'], dtype=object)

In [56]:
# Least expensive drink of each type
drinks_discounted.groupby('Drink').agg(min)

,Cafe,Discounted_Price
Drink,,
Coffee,Nefeli,1.5
Espresso,Nefeli,1.0
Milk Tea,Tea One,3.0


In [61]:
# Cheapest, post-discount drink at each Cafe
drinks_discounted.groupby('Cafe').agg(min)

,Drink,Discounted_Price
Cafe,,
Nefeli,Coffee,1.0
Tea One,Milk Tea,3.0


### old Midterm questions

In [60]:
# Read table from csv
trips = pd.read_csv('data/trip.csv')
trips

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
3,913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
...,...,...,...,...,...,...,...,...,...,...,...
354148,432950,6712,9/1/2014 3:16,Harry Bridges Plaza (Ferry Building),50,9/1/2014 5:08,San Francisco Caltrain (Townsend at 4th),70,259,Customer,44100
354149,432949,538,9/1/2014 0:05,South Van Ness at Market,66,9/1/2014 0:14,5th at Howard,57,466,Customer,32
354150,432948,568,9/1/2014 0:05,South Van Ness at Market,66,9/1/2014 0:15,5th at Howard,57,461,Customer,32
354151,432947,569,9/1/2014 0:05,South Van Ness at Market,66,9/1/2014 0:15,5th at Howard,57,318,Customer,32


In [84]:
# Name of the station where the most rentals ended (assuming no ties)
trips['End Station'].value_counts().index[0]

'San Francisco Caltrain (Townsend at 4th)'

In [110]:
# Name of the station where the most rentals ended (with possibility of ties)
(trips['End Station']
 .value_counts()
 .loc[trips['End Station'].value_counts() == 
      trips['End Station'].value_counts()[0]]
 .index.values)

array(['San Francisco Caltrain (Townsend at 4th)'], dtype=object)

In [160]:
# The number of stations for which the average duration ending at the station
# was more than 10 min, after removing durations over 1800 seconds
commute = trips.loc[trips.Duration < 1800]

np.sum(commute
 .groupby('End Station')
 .Duration.agg(np.mean) > (10 * 60)
)


21

In [161]:
x = 2
x < 1

False

In [162]:
1 < x < 5

True

### Advanced Where

In [173]:
data_url = ('http://www2.census.gov/programs-surveys/popest/datasets/'
            '2010-2015/national/asrh/nc-est2015-agesex-res.csv')
full_census_table = pd.read_csv(data_url)
partial = full_census_table.loc[:, ['SEX', 'AGE', 'POPESTIMATE2010', 
                                    'POPESTIMATE2015']]
us_pop = partial.rename({'POPESTIMATE2010':'2010', 'POPESTIMATE2015':'2015'}, 
                        axis='columns')
us_pop

,SEX,AGE,2010,2015
0,0,0,3951330,3978038
1,0,1,3957888,3968564
2,0,2,4090862,3966583
3,0,3,4111920,3974061
...,...,...,...,...
302,2,98,37532,47272
303,2,99,26074,34064
304,2,100,45058,61886
305,2,999,157258820,163189523


In [174]:
# Rows where age equals 70
us_pop.loc[us_pop.AGE == 70]

,SEX,AGE,2010,2015
70,0,70,2062577,2492490
172,1,70,954073,1162672
274,2,70,1108504,1329818


In [175]:
# Rows where age equals 70 and sex does not equal 0
us_pop.loc[(us_pop.AGE == 70) & (us_pop.SEX != 0)]

,SEX,AGE,2010,2015
172,1,70,954073,1162672
274,2,70,1108504,1329818


In [178]:
# rows where ratio of 2015 to 2010 population is greater than 1.5
us_pop.loc[(us_pop['2015'] / us_pop['2010']) > 1.5]

,SEX,AGE,2010,2015
196,1,94,43827,68135
197,1,95,31736,48015
199,1,97,14775,23092
200,1,98,9505,14719
201,1,99,6104,9577
202,1,100,9352,15088


In [182]:
# Average duration of trips that begin and end at the same terminal
np.mean(commute.Duration.loc[commute['Start Terminal'] == commute['End Terminal']])

758.612993783838

In [184]:
# Average duration of trips that don't begin and end at the same terminal
np.mean(commute.Duration.loc[commute['Start Terminal'] != commute['End Terminal']])

546.880608118648